# EKM Knowledge Exploration Notebook

This notebook explores the Episodic Knowledge Mesh and demonstrates its capabilities.

In [ ]:
# Import EKM components
import asyncio
import numpy as np
from ekm.core.mesh import EKM
from ekm.storage.sql_storage import SQLStorage
from ekm.providers.openai import OpenAIProvider

# Initialize EKM components
storage = SQLStorage("sqlite:///./ekm_notebook.db")
llm = OpenAIProvider(api_key="YOUR_API_KEY")  # Replace with your actual API key
embeddings = OpenAIProvider(api_key="YOUR_API_KEY")  # Replace with your actual API key

# Create EKM instance
ekm = EKM(storage, llm, embeddings)

In [ ]:
# Train the EKM with sample text
sample_text = """
Artificial intelligence (AI) is intelligence demonstrated by machines, in contrast to the natural intelligence displayed by humans and animals. Leading AI textbooks define the field as the study of "intelligent agents": any device that perceives its environment and takes actions that maximize its chance of successfully achieving its goals.

Colloquially, the term "artificial intelligence" is often used to describe machines that mimic "cognitive" functions that humans associate with the human mind, such as "learning" and "problem solving".
"""

# Create a workspace and train
workspace_id = "ai_research_workspace"

# Train the EKM with the sample text
await ekm.train(workspace_id, sample_text, title="AI Introduction")
print(f"Trained EKM with workspace ID: {workspace_id}")

In [ ]:
# Retrieve information from the EKM
query = "What is artificial intelligence?"

# Perform retrieval
results = await ekm.retrieve(workspace_id, query)

print(f"Query: {query}")
print(f"Results: {len(results['results'])} items retrieved")
for i, result in enumerate(results['results'][:3]):  # Show top 3 results
    print(f"Result {i+1}: {result['content'][:100]}...")

In [ ]:
# Explore attention mechanisms
from ekm.core.attention import QKVAttentionRetriever

# Create attention retriever
attention_retriever = QKVAttentionRetriever(embedding_dim=768, num_heads=8)

# Get embeddings for the query
query_embedding = await embeddings.embed_query(query)

# Retrieve with attention details
attention_results = await attention_retriever.retrieve(
    storage=storage,
    workspace_id=workspace_id,
    query_embedding=query_embedding,
    top_k=3
)

print("Attention weights breakdown:")
interpretations = attention_retriever.interpret_attention_weights(attention_results)
for key, value in interpretations.items():
    print(f"  {key}: {value}")

In [ ]:
# Visualize knowledge graph (if available)
try:
    from ekm.core.attention_viz import AttentionDebugger
    
    debugger = AttentionDebugger()
    debug_info = debugger.create_debug_info(
        query=query,
        gku_names=["AI Concepts"],  # Example GKU names
        attention_result=attention_results
    )
    
    print("Attention visualization created successfully!")
    # Note: Actual visualization would depend on the debugger implementation
except ImportError:
    print("Attention visualization not available")

In [ ]:
# Cleanup
print("Notebook execution completed!")